In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
df_fake = pd.read_csv(r"C:\Users\DELL\Downloads\archive (1)\Fake.csv")
df_true = pd.read_csv(r"C:\Users\DELL\Downloads\archive (1)\True.csv")

In [3]:
df_fake["label"] = "FAKE"
df_true["label"] = "REAL"

In [4]:
df = pd.concat([df_fake, df_true], axis=0)
df = df[["text", "label"]]  

In [5]:
df = df.sample(frac=1).reset_index(drop=True)


In [6]:
df.to_csv("dataset.csv", index=False)


In [7]:
X = df["text"]
y = df["label"].map({"REAL": 0, "FAKE": 1})

In [8]:
vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
X_vect = vectorizer.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)


In [10]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [11]:
pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))

In [12]:
print("✅ Model and vectorizer saved!")


✅ Model and vectorizer saved!


In [13]:
import streamlit as st
import pickle

In [14]:
model = pickle.load(open("model.pkl", "rb"))
vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))

In [18]:
st.set_page_config(page_title="Fake News Detector")
st.title("📰 Fake News Detection")
st.write("Enter a news article below to check if it's **Fake** or **Real**.")
user_input = st.text_area("📝 News Article Text", height=200)


if st.button("🔍 Predict"):
    if not user_input.strip():
        st.warning("Please enter some text first.")
    else:
        input_vect = vectorizer.transform([user_input])
        prediction = model.predict(input_vect)[0]

        if prediction == 1:
            st.error("🚫 This looks like FAKE News!")
        else:
            st.success("✅ This appears to be REAL News!")